In [1]:
import networkx as nx;
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def InnerProblem(Gamma, G, s, t, A):
    
    X = {};
    Beta = {};
    Theta = {};
    
    sub = gp.Model("Sub");
    
    x = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = sub.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    beta = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sub.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
               gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
    
    sub.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        sub.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-Gamma[i,j]) <= 0);
        sub.addConstr(beta[i,j] >= theta[i,j] - Gamma[i,j]);
        sub.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        if G.edges[i,j]['special'] == 0:
            sub.addConstr(z[i,j] == 0);
        else:
            sub.addConstr(x[i,j] - (1/A)*z[i,j] >= 0);
    
    sub.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    sub.setObjective(gp.quicksum(z[i,j] for i,j in G.edges), GRB.MAXIMIZE);
    
    #sub.setParam("IntegralityFocus",1);
    #sub.setParam("NumericFocus",2);       
    #sub.setParam('TimeLimit', T_Lim);
            
    sub.update();
    sub.setParam("OutputFlag", 0);
    sub.optimize();
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Beta[i,j] = beta[i,j].x;
        Theta[i,j] = theta[i,j].x;
        
    
    return X, Beta, Theta, sub.ObjVal;

In [3]:
def Sub(Gamma, G, s, t, A, M1, M2):
    
    X = {};
    Beta = {};
    Theta = {};
    
    sub = gp.Model("Sub");
    
    x = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = sub.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    beta = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sub.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
               gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
    
    sub.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        sub.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-Gamma[i,j]) <= 0);
        sub.addConstr(beta[i,j] >= theta[i,j] - Gamma[i,j]);
        sub.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        if G.edges[i,j]['special'] == 0:
            sub.addConstr(z[i,j] == 0);
        else:
            sub.addConstr(x[i,j] - (1/A)*z[i,j] >= 0);
    
    sub.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    sub.setObjective(gp.quicksum(z[i,j] for i,j in G.edges)
                     - M1*gp.quicksum(x[i,j]*Gamma[i,j] for i,j in G.edges)
                     - M2*gp.quicksum((theta[i,j] - beta[i,j])*(1-Gamma[i,j]) for i,j in G.edges), GRB.MAXIMIZE);
    
    #sub.setParam("IntegralityFocus",1);
    #sub.setParam("NumericFocus",2);       
    #sub.setParam('TimeLimit', T_Lim);
            
    sub.update();
    sub.setParam("OutputFlag", 0);
    sub.optimize();
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Beta[i,j] = beta[i,j].x;
        Theta[i,j] = theta[i,j].x;
        
        #print('beta(%d,%d): %g'%(i,j,Beta[i,j]))
        #print('Theta(%d,%d): %g'%(i,j,Theta[i,j]))
        
        
    
    return X, Beta, Theta, sub.ObjVal;

In [ ]:
# MAIN


n_Networks = 10;
Budget = [3];

T_Limit = 7200;



summaryName = "Benders_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance,Budget,LB,UB,Gap,N_Iterations,Time\n');
file_summary.close();


for n in range(1, n_Networks+1):
    network = 'Net'+str(n);
    
    start_time = time.time(); 
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
    A = 0;
    U = G.edges[t,s]['capacity'];
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
    
    M1 = A*A;
    M2 = A;
    
    UB = A;
    LB = 0;
    
    sub_ObjVal = 0;
    master_ObjVal = 0;
    
    X = {};
    Beta = {};
    Theta = {};
    
    tol = 0.01;
    
    k = 0;
    Gamma = {};
    
    for i,j in G.edges:
        Gamma[i,j] = 0;
    
    for budget in Budget:    

        start_time = time.time();
        ite_time = time.time();
        
        master = gp.Model("Master");
        gamma = master.addVars(G.edges, vtype=GRB.BINARY);
        eta = master.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = A);
        
        master.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget); 
        
        master.setObjective(eta, GRB.MINIMIZE);
        
        master.setParam("IntegralityFocus",1);
        #master.setParam("NumericFocus",2);       
        #master.setParam('TimeLimit', T_Lim);
        
        print('\n%s budget %g \n\n' %(network, budget));
        
        X, Beta, Theta, UB = Sub(Gamma, G, s, t, A, M1, M2);         
        
        file = open('Benders_'+network+'_b'+str(budget)+'.txt', "w");
        
        instance = 'LogFile_Benders_'+network+'_b'+str(budget)+'.txt';
        
        while (UB - LB) > tol and (ite_time - start_time) < T_Limit:
            k += 1;
             
            master.addConstr(eta >= UB - M1*gp.quicksum(X[i,j]*gamma[i,j] for i,j in G.edges)
                             - M2*gp.quicksum((Theta[i,j] - Beta[i,j])*(1-gamma[i,j]) for i,j in G.edges));
            
            master.setParam("LogToConsole", 0);
            master.setParam("OutputFlag", 1);
            master.setParam("LogFile", instance);
            
            master.update();
            master.optimize();
            
            LB = master.ObjVal
        
            for i,j in G.edges:
                Gamma[i,j] = gamma[i,j].x;
                if Gamma[i,j] > 0.001:
                    print('arc (%d,%d)'%(i,j));
            
            X, Beta, Theta, UB = Sub(Gamma, G, s, t, A, M1, M2);
            
            print('Iterations %g' %k);
            print('Upper Bound: %g' %UB);
            print('Lower Bound: %g\n' %LB);
            
            ite_time = time.time();
        
        if LB != 0:
            gap = abs(UB-LB)/abs(LB);
        else:
            gap = 100000;
            
        end_time = time.time();
        run_time = round(end_time - start_time,2)
        now = datetime.now();  
        
        file.write('Instance executed at: %s \n\n' %now.strftime("%c"));
        file.write('Runtime: %g secs \n' %run_time);
        
        file.write('Number of iterations: %g \n' %k);
        file.write('Upper Bound: %g\n' %UB);
        file.write('Lower Bound: %g\n' %LB);
        
        file.write('\nInterdiction:\n');
        for i,j in G.edges:
            if Gamma[i,j] > 0.001:
                file.write('Arc: (%d,%d)\n'%(i,j));
            
        file.close();
        
        rowFields = [network, budget,LB,UB,gap,k,run_time];
    
        with open(summaryName, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile);
            csvwriter.writerow(rowFields);
            csvfile.close();

            

Set parameter TokenServer to value "license8.clemson.edu"
Set parameter IntegralityFocus to value 1

Net1 budget 3 


arc (1,6)
arc (1,7)
arc (4,9)
Iterations 1
Upper Bound: 13
Lower Bound: 0

arc (3,10)
arc (13,16)
arc (14,17)
Iterations 2
Upper Bound: 15
Lower Bound: 0

arc (13,19)
arc (13,20)
arc (14,16)
Iterations 3
Upper Bound: 16
Lower Bound: 0

arc (7,14)
arc (15,16)
arc (15,18)
Iterations 4
Upper Bound: 16
Lower Bound: 0

arc (2,7)
arc (14,20)
arc (11,20)
Iterations 5
Upper Bound: 15
Lower Bound: 0

arc (10,12)
arc (13,17)
arc (14,18)
Iterations 6
Upper Bound: 16
Lower Bound: 0

arc (3,9)
arc (4,10)
arc (8,12)
Iterations 7
Upper Bound: 14
Lower Bound: 0

arc (15,20)
Iterations 8
Upper Bound: 16
Lower Bound: 0

arc (2,7)
arc (4,10)
arc (10,14)
Iterations 9
Upper Bound: 14
Lower Bound: 0

arc (7,13)
arc (8,13)
arc (15,18)
Iterations 10
Upper Bound: 16
Lower Bound: 0

arc (7,13)
arc (8,12)
arc (13,16)
Iterations 11
Upper Bound: 16
Lower Bound: 0

arc (3,9)
arc (3,10)
arc (10,14)
I